# Rotating marker data

> Alex Woodall <br>
> Auckland Bioengineering Institute <br>
> University of Auckland, New Zealand

This is an example code to read in markers from a TRC file using the _trc_read_ function, rotate them to an OpenSim coordinate systemand then write then back to a new _.trc_ file.

Start by importing the libraries needed to run the task.

In [1]:
import numpy as np
import os

import sys
sys.path.insert(1, r'./../Functions')  # add to path

from read_trc import read_trc
from write_trc import write_trc

## Reading in a TRC file
We can read in the desired trc file, containing the coordinate data for each of the markers during a motion capture recording session.

In [2]:
# Read in the trc file
mkr_data, file_path = read_trc()

## Extracting data

The TRC file is structured to have information about the recording, and the data. We will need to extract the information to get the header files and the data to get the markers to rotate.

In [3]:
# Pull out header information from the trc file
start_frame = mkr_data["Information"]["OrigDataStartFrame"]
frame_rate = mkr_data["Information"]["CameraRate"]
n_markers = int(mkr_data["Information"]["NumMarkers"])
file_name = mkr_data["Information"]["FileName"]
num_frames = int(mkr_data["Information"]["NumFrames"])
units = mkr_data["Information"]["Units"]

In [4]:
# Pull out data from the trc file
marker_data = mkr_data["Data"]["RawData"]
marker_labels = mkr_data["Data"]["MarkerLabels"]

frame_numbers = mkr_data["Data"]["FrameNums"]

time = mkr_data["Data"]["Time"] # Get time array

We need to make sure that the time array is correct. We can use the frame rate and start frame from the information data to set this.

In [5]:
# Ensure the time array starts at the first point
time_diff = 1/frame_rate

start_time = start_frame * time_diff
end_time = num_frames * time_diff + start_time - time_diff

new_time = np.linspace(start_time, end_time, num_frames) # TODO fix this problem

## Rotating the data

We can now rotate the data 90° about the x-axis, so that the y-axis is the vertical axis.

In [6]:
# Rotate + 90deg about X, so that Y is the vertical axis (consistant with OpenSim)
rot = np.array([(1, 0, 0), (0, 0, 1), (0, -1, 0)]) # Rotation matrix

rotated_marker_data = np.zeros(np.shape(marker_data))

# Rotate the marker data
for i in range(n_markers):
    rotated_marker_data[:,(3*(i+1)-3):3*(i+1)] = np.matmul(rot, marker_data[:, (3*(i+1)-3):3*(i+1)].T).T

Once rotated, we also need to append the frame numbers (1, 2, 3..., n) and the time data to the marker data

In [7]:
# Append frame and time column to the front of the rotated marker data
new_mkr_data = np.concatenate((frame_numbers[:, np.newaxis], new_time[:, np.newaxis], rotated_marker_data),axis=1)

## Writing to a file

With the rotation done, we can now write the information and data back to a new _.trc_ file.

In [8]:
# Create new file name and write to a trc
new_file_name = file_path.replace(".trc", "_rotated.trc")

write_trc(marker_labels, mkr_data["Information"], frame_numbers, new_mkr_data, new_file_name)

C:/Users/alexw/Dropbox/ABI/BIOMENG 771 Files/Tutorial 1/MATLABcode/ReadTRCfile/jw_ngait_og1_rotated.trc
